## PAIR PROGRAMMING ETL I

### Extracción -API's
---

In [1]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np

Vamos a nutrir los datos de los ataques de los tiburones con información climática de los paises que tenemos. Para eso vamos a usar la API del clima que hemos aprendido en la clase invertida:

In [2]:
url = f'http://www.7timer.info/bin/api.pl?lon=-{longitud}&lat={latitud}&product={producto}&output=json'

NameError: name 'lon' is not defined

En este caso os recomendamos que uséis el producto meteo para obtener la información climática. Para hacer la llamada a la API necesitamos también las coordenadas de los paises que tenemos en el dataset.

Por lo tanto, el objetivo es que saquéis la información del clima de la API para los paises que tenemos. Pero antes de poneros manos a la obra, tenemos muchísimos países y esto puede hacerse eterno. Sacad la información climática solo para los siguientes países, con las siguientes coordenadas:

In [3]:
pais = ['USA', 'Australia', 'South Africa', 'New Zealand', 'Papua New Guinea']
latitud= [39.7837304, -24.7761086, -28.8166236, -41.5000831, -5.6816069]
longitud = [-100.445882, 134.755, 24.991639,172.8344077,144.2489081]
lon_lat 
producto = 'civil'

![alt text]('../anamor/ADALAB/PairProgramming/imagenpairETL.png')

Requisitos de este ejercicio de pair programming:
- Deberéis meter toda la información climática en un único dataframe.
- Deberéis hacer la llamada a la API de una sola vez. Es decir, tendréis que iterar por la lista de países y sacar la información del clima para cada uno de ellos.
- Al meter toda la información en un único dataframe tendremos que crear una columna que corresponda con el nombre del país.
- Guardar el dataframe obtenido para usarlo en la siguiente sesión de pair programming.

💡 Pistas 💡:
- Crearos un diccionario donde:
    - Las keys sean los nombres de los paises
    - Los values sean las coordenadas de los paises en formato lista.

- Iterar por el diccionario. Dentro del for haremos la llamada a la API como hemos aprendido en las clases invertidas. Recordamos que para iterar por el diccionario tenemos que seguir la siguiente estructura: 
*_for key, value in diccionario.items(): 
print(key, value)_*

- Crearnos un dataframe vacío donde iremos concatenando los datos obtenidos de la API cada país.
- Guardad el csv ya que mañana lo necesitaremos para hacer el ejercicio de pair programming.

In [11]:
diccionario = list(zip(pais,longitud,latitud))# reveer si podemos sacarlo con el zip para no hacerlo de forma manual
diccionario

[('USA', -100.445882, 39.7837304),
 ('Australia', 134.755, -24.7761086),
 ('South Africa', 24.991639, -28.8166236),
 ('New Zealand', 172.8344077, -41.5000831),
 ('Papua New Guinea', 144.2489081, -5.6816069)]

In [40]:
dicc_paises= {'USA': [-100.445882, 39.7837304],'Australia': [134.755, -24.7761086],'South Africa': [24.991639, -28.8166236],'New Zealand': [172.8344077, -41.5000831],'Papua New Guinea': [144.2489081, -5.6816069]}

In [19]:
for key, value in dicc_paises.items():

    print(value)

[-100.445882, 39.7837304]
[134.755, -24.7761086]
[24.991639, -28.8166236]
[172.8344077, -41.5000831]
[144.2489081, -5.6816069]


In [23]:
dic= {'USA': [-100.445882, 39.7837304] }


#lon = -100.445882
#lat = 39.7837304
#producto = 'civil'

In [25]:
dic["USA"][1]

39.7837304

In [35]:
url =f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'

response = requests.get(url=url)

# pedimos a la API que nos diga si lo estamos haciendo bien con el método status_code
response.status_code

# le pedimos que nos diga el por qué
response.reason
#datos= pd.DataFrame()
datos= pd.json_normalize(response.json()['dataseries'])

datos['country']=key

200

'OK'

In [36]:
datos

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,country
0,3,1,15,none,0,12,43%,clearnight,W,2,Papua New Guinea
1,6,1,15,none,0,12,38%,clearnight,N,2,Papua New Guinea
2,9,1,15,none,0,12,33%,clearday,NW,2,Papua New Guinea
3,12,1,15,none,0,14,28%,clearday,NE,2,Papua New Guinea
4,15,1,15,none,0,14,36%,clearday,E,2,Papua New Guinea
...,...,...,...,...,...,...,...,...,...,...,...
59,180,1,15,none,0,13,30%,clearday,SW,2,Papua New Guinea
60,183,2,10,none,0,13,32%,clearday,NE,2,Papua New Guinea
61,186,2,10,none,0,14,42%,clearnight,W,2,Papua New Guinea
62,189,1,15,none,0,13,40%,clearnight,W,2,Papua New Guinea


In [48]:
for key, value in dicc_paises.items(): 
    print(value)[0]
    #for i in value:
        #print(i)

[-100.445882, 39.7837304]


TypeError: 'NoneType' object is not subscriptable

In [49]:
for key, value in dicc_paises.items(): 

 
    url =f'http://www.7timer.info/bin/api.pl?lon=-{dicc_paises[key][0]}&lat={dicc_paises[key][1]}&product={producto}&output=json'

# hacemos la petición de los datos a la API
response = requests.get(url=url)

clima_paises = pd.json_normalize(response.json()['dataseries'])

clima_paises['country']=key

df_clima = pd.concat(clima_paises), axis=0, ignore_index = True)
# pedimos a la API que nos diga si lo estamos haciendo bien con el método status_code
response.status_code

# le pedimos que nos diga el por qué
response.reason

SyntaxError: cannot assign to function call (1242005001.py, line 13)

In [50]:
clima_paises['country'].unique()

array(['Papua New Guinea'], dtype=object)